### Libraries

In [14]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [15]:
data=pd.read_csv('imdb_dataset.csv')

In [16]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [17]:
data.shape

(50000, 2)

In [18]:
type(data)

pandas.core.frame.DataFrame

In [19]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [20]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [21]:
# One Hot Encoding
# label Encoding

In [22]:
# Positive -> 1
# Negative -> 0
data.replace({"sentiment":{"positive":1, "negative": 0}}, inplace = True)

In [23]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


### LSTM -> Long Short Term Memory

In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [26]:
train_data, test_data = train_test_split(data, test_size= 0.2, random_state=42)

In [27]:
train_data.shape

(40000, 2)

In [28]:
test_data.shape

(10000, 2)

In [29]:
tokenizer = Tokenizer(num_words = 5000)
test = tokenizer.fit_on_texts(train_data['review'])

In [30]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [31]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]])

In [32]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]])

In [33]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [34]:
Y_train

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64

### LSTM Model Building

In [41]:
model = Sequential()
model.add(Embedding(input_dim = 5000, output_dim = 128, input_length = 200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [43]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])

In [45]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 81s 160ms/step - accuracy: 0.9286 - loss: 0.1903 - val_accuracy: 0.8919 - val_loss: 0.2706
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 78s 156ms/step - accuracy: 0.9399 - loss: 0.1670 - val_accuracy: 0.8896 - val_loss: 0.2761
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 79s 158ms/step - accuracy: 0.9453 - loss: 0.1550 - val_accuracy: 0.8903 - val_loss: 0.2831
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 78s 157ms/step - accuracy: 0.9527 - loss: 0.1333 - val_accuracy: 0.8895 - val_loss: 0.2965
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 79s 157ms/step - accuracy: 0.9594 - loss: 0.1185 - val_accuracy: 0.8863 - val_loss: 0.3252


In [46]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.8926 - loss: 0.3006


In [47]:
print(loss)

0.30325037240982056


In [48]:
print(accuracy)

0.8928999900817871


In [50]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

### Building Predictive System

In [51]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [57]:
predictive_system("This movie is not good just awesome ")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


'positive'

In [58]:
predictive_system("Time waste")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


'negative'

In [59]:
model.save("model.h5")